In [16]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import sys
sys.path.append('../..')
from functions.db_inserts import *
from functions.aws_functions import *
from functions.select_cols_functions import *

In [18]:
env_mode = 'prd' # 'dev' ou 'prd'

### Lendo arquivo fonte

In [19]:
bucket_name = "adoptai-raw-dev"

In [ ]:
if env_mode == 'dev':
    df_trusted = pd.read_csv('../../../data/train_2_classes.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = ['train_2_classes.csv', df_trusted]
elif env_mode == 'prd':
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Parametros de gravação

In [20]:
nome_fonte = s3[0]
nome_modelo = 'Random Forest'
parameters = {'n_estimators': [1, 10, 100, 500, 1000]}

In [21]:
list(parameters.keys())[0]

'n_estimators'

### Preparativos para executar o modelo

In [22]:
# Separar as features (X) e os rótulos (y)
x = df_trusted.iloc[:, :-1]
y = df_trusted.iloc[:, -1]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Rodando modelo Random Forest

In [24]:
accuracy = []
precision = []
recall = []
f1 = []

In [25]:
for n_estimator in parameters['n_estimators']:

    # Treinar o modelo Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimator, random_state=42)
    start_time = time.time()
    rf.fit(x_train, y_train)

    # Fazer previsões
    y_pred = rf.predict(x_test)

    # Avaliar o modelo
    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average=None))
    recall.append(recall_score(y_test, y_pred, average=None))
    f1.append(f1_score(y_test, y_pred, average=None))

In [26]:
end_time = time.time()
training_time = end_time - start_time

### Obtendo métricas de desempenho

In [27]:
metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

### Gravando dados no banco

In [28]:
model = {'nome_modelo': nome_modelo, 'nome_fonte': nome_fonte}
execution = {'accuracy': accuracy, 'start_time': start_time, 'end_time': end_time}
hyperparams = parameters
desempenho = {'precision_values': metrics['precision'], 'recall_values': metrics['recall'], 'f1_values': metrics['f1_score']}

In [29]:
insert_values(model, execution, hyperparams, desempenho, env=env_mode)

Executando em dev
Registros inseridos
